## Basic example of particle swarm to optimize a one-variable function.

In [5]:
import pandas as pd
import networkx as nx
import random

In [17]:
# Generate the network topology - in this case, von Neumann (i.e. every node has exactly 4 neighbors)
pop_size = 40
num_neighbors = 4
graph = nx.random_regular_graph(num_neighbors, pop_size)

In [18]:
#Set the sociality (how much the node cares about the group opinion), memory(how much it cares about its personal best),
#and stability (the tendency to stay where it is)
sociality = 0.5
memory = 0.3
stability = 1 - sociality - memory

In [19]:
def f(x):
    return x**2

In [20]:
# Set initial population - random numbers btween -100 and 100
initial_values = [200*random.random() - 100 for _ in range(pop_size)]

In [21]:
Population = pd.DataFrame({'node' : list(range(40)), 'current_value' : initial_values, 'personal_best' : initial_values})

In [22]:
Population['fitness'] = Population.current_value.apply(f)

In [23]:
Population['neighbors'] = Population.node.apply(lambda x: [x] + [_ for _ in graph[x]])

In [24]:
# Find the group best - this is ugly and maybe could be cleaned up to not look so bad
Population['group_best'] = Population.node.apply(lambda x:
Population.loc[(Population.node.isin(Population.loc[x].neighbors)) &  
               (Population.fitness == Population.loc[Population.node.isin(Population.loc[x].neighbors)].fitness.min())].values[0][1])
Population['group_best_fitness'] = Population.group_best.apply(f)
Population['personal_best_fitness'] = Population.personal_best.apply(f)

In [25]:
print(Population.fitness.min())

14.30529730276412


In [26]:
def mutation(current_value):
    x = random.random()
    if x < .05: return 200*random.random() - 100
    return current_value

In [27]:
num_iterations = 1000
for i in range(num_iterations):
    #Apply crossover
    Population.current_value = Population.apply(lambda row: stability*row.current_value + memory*row.personal_best + sociality*row.group_best, axis = 1)
    Population.current_value = Population.current_value.apply(mutation)
    Population.fitness = Population.current_value.apply(f)
    Population.personal_best = Population.apply(lambda row: row.current_value if row.fitness < row.personal_best_fitness else row.personal_best, axis = 1)
    Population['personal_best_fitness'] = Population.personal_best.apply(f)
    # Find the group best - this is ugly and maybe could be cleaned up to not look so bad
    Population['group_best'] = Population.node.apply(lambda x:
    Population.loc[(Population.node.isin(Population.loc[x].neighbors)) &  
               (Population.personal_best_fitness == Population.loc[Population.node.isin(Population.loc[x].neighbors)].personal_best_fitness.min())].values[0][2])
    Population['group_best_fitness'] = Population.group_best.apply(f)

    print(Population.fitness.min(), Population.fitness.mean())

3.415491193866568 1270.967284598431
3.411455572652617 383.73732712744004
0.17621660255423766 377.06514118377834
0.016696149820981457 256.3606890758583
0.01269807814021536 17.06787250609812
0.010516413828622126 92.36150879865487
1.3467501832002454e-06 4.167232781631296
1.3467501832002454e-06 157.44302264729686
1.3467501832002454e-06 6.411620515862173
1.3467501832002454e-06 21.810242301964852
1.3467501832002454e-06 160.37517459320796
4.407677137479679e-07 138.82588801163837
3.272321770338028e-07 264.0682328180422
2.1096237200671845e-09 19.682875187265957
2.6679377084764336e-11 29.93364800407354
2.6679377084764336e-11 119.45087154418539
2.6679377084764336e-11 255.10327213348063
2.2513590359617462e-11 117.6149538529776
2.2513590359617462e-11 94.67511438806477
9.806798084704491e-12 68.94093419978796
3.6777131129805533e-13 121.53559587514523
3.6777131129805533e-13 86.09381765482844
9.293600108169132e-18 230.37580287245805
9.293600108169132e-18 306.07137788328845
9.293600108169132e-18 12.2428

KeyboardInterrupt: 